<a href="https://colab.research.google.com/github/gkdivya/EVA/blob/main/3_PyTorchNeuralNetwork/MNIST_RandomNumber_Addition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Addition of two numbers using neural network

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
import numpy as np

import random

In [2]:
## Creation of Random MNIST dataset

from torch.utils.data import Dataset, TensorDataset
import random

class RandomMNISTDataset(Dataset):
  def __init__(self, MNISTDataset):
    self.MNISTDataset = MNISTDataset

  def __getitem__(self, index):
    image = self.MNISTDataset[index][0]
    label = self.MNISTDataset[index][1]
    randomNo = random.randint(0,9)
    one_hotrandomNo = torch.nn.functional.one_hot(torch.arange(0, 10))
    sum = label + randomNo
    return image, label, one_hotrandomNo[randomNo], sum

  def __len__(self):
    return len(self.MNISTDataset)

In [ ]:
torch.nn.functional.one_hot(torch.arange(0, 10))

In [3]:
# torchvision.datasets.MNIST outputs a set of PIL images
# We transform them to tensors
transform = transforms.ToTensor()

# Load and transform data
MNIST_trainset = torchvision.datasets.MNIST('/tmp', train=True, download=True, transform=transform)
MNIST_testset = torchvision.datasets.MNIST('/tmp', train=False, download=True, transform=transform)

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting /tmp/MNIST/raw/train-images-idx3-ubyte.gz to /tmp/MNIST/raw




Extracting /tmp/MNIST/raw/train-labels-idx1-ubyte.gz to /tmp/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting /tmp/MNIST/raw/t10k-images-idx3-ubyte.gz to /tmp/MNIST/raw




Extracting /tmp/MNIST/raw/t10k-labels-idx1-ubyte.gz to /tmp/MNIST/raw

Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [4]:
train_dataset = RandomMNISTDataset(MNIST_trainset)
test_dataset = RandomMNISTDataset(MNIST_testset)

In [5]:
from torch.utils.data import DataLoader
train_loader = DataLoader(train_dataset,batch_size=128,shuffle=True)
test_loader = DataLoader(test_dataset,batch_size=128)

### Visualization of a batch

In [6]:
def show_batch(batch):
    im = torchvision.utils.make_grid(batch)
    plt.imshow(np.transpose(im.numpy(), (1, 2, 0)))

dataiter = iter(train_loader)
images, labels, randomNumbers, sums = dataiter.next()

print('Images: ', images.shape)
print('Labels: ', labels.shape)
print('Random Numbers: ', randomNumbers.shape)
print('Sum: ', sums.shape)

print('Labels: ', labels)
print('Random Numbers: ', randomNumbers)
print('Sum: ', sums)



Images:  torch.Size([128, 1, 28, 28])
Labels:  torch.Size([128])
Random Numbers:  torch.Size([128, 10])
Sum:  torch.Size([128])
Labels:  tensor([2, 0, 9, 8, 9, 4, 1, 3, 0, 2, 1, 7, 1, 3, 1, 3, 3, 7, 3, 9, 9, 7, 2, 6,
        7, 7, 5, 4, 6, 5, 5, 8, 7, 4, 9, 7, 3, 9, 6, 2, 7, 2, 4, 0, 6, 5, 8, 8,
        1, 9, 2, 3, 1, 8, 1, 6, 2, 2, 7, 7, 1, 3, 1, 3, 0, 4, 4, 3, 1, 2, 3, 7,
        1, 9, 0, 9, 7, 3, 8, 2, 3, 3, 0, 1, 9, 3, 3, 3, 8, 6, 1, 5, 1, 2, 7, 7,
        3, 1, 6, 3, 0, 7, 7, 2, 1, 8, 5, 6, 1, 0, 6, 2, 2, 3, 3, 4, 0, 8, 1, 7,
        4, 3, 3, 4, 0, 1, 7, 0])
Random Numbers:  tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 1, 0, 0],
        [0, 0, 1,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 1, 0],
        [0, 0, 0,  ..., 0, 0, 1],
        [0, 0, 0,  ..., 0, 0, 0]])
Sum:  tensor([ 5,  7, 11, 12, 10,  5,  1,  8,  6,  6,  6,  7,  1, 12,  2, 10, 10,  9,
         3, 18, 13, 11,  8, 10, 13, 12, 10, 10,  8, 10, 10, 16, 10,  4, 10,  8,
        12, 14,  8,  2,  9,  

### Defining Network

In [7]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)       #Input: 28*28*1    Output:28 * 28 * 32    GRF:3 * 3  (GRF - Global Receptive Field)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)      #Input: 28*28*32   Output:28 * 28 * 64    GRF:5 * 5
        self.pool1 = nn.MaxPool2d(2, 2)                   #Input: 28*28*64   Output:14 * 14 * 64    GRF:10*10 (for now we are considering that with max pooling, receptive field doubles - but this is not entirely correct)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)     #Input: 14*14*64   Output:14 * 14 * 128   GRF:12*12
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)    #Input: 14*14*128  Output:14 * 14 * 256   GRF:14*14
        self.pool2 = nn.MaxPool2d(2, 2)                   #Input: 14*14*256  Output: 7 * 7 * 256    GRF:28*28
        self.conv5 = nn.Conv2d(256, 512, 3)               #Input: 7*7*256    Output: 5 * 5 * 12     GRF:30*30
        self.conv6 = nn.Conv2d(512, 1024, 3)              #Input: 5*5*12     Output: 3 * 3 * 1024   GRF:32*32
        self.conv7 = nn.Conv2d(1024, 10, 3)               #Input: 3*3*1024   Output: 1 * 1 * 10     GRF:34*34

        #self.fc_random = nn.Linear(128, 10)
        self.fc1 = nn.Linear(10+10, 128)
        self.fc2 = nn.Linear(128, 30)
        self.fc3 = nn.Linear(30, 19)

    def forward(self, image, randomNumber):
        #The first convolutional layer self.conv1 has a convolutional operation on input tensor x, followed by a relu activation operation 
        #whcih is then passed to second convolution operation self.conv2 followed by a relu whose output is then passed to a max pooling 
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(image)))))

        #The output of the max pool operation is passed to another two convolution and relu activation operation followed by max pooling. 
        #The relu() and the max_pool2d() calls are just pure operations. Neither of these have weights  
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))

        #The output of the max pool operation from 4th convolution is passed to another two sets of convolution and relu activation operation
        x = F.relu(self.conv6(F.relu(self.conv5(x))))

        # The output from 6th convolution is then passed to a linear layer
        x = self.conv7(x)  
        
        #The flatten's all of the tensor's elements into a single dimension.
        x = x.view(-1, 10)
        
        #randomNumber = F.linear(self.fc_random(randomNumber))
        #The flatten's all of the tensor's elements into a single dimension.
        x1 = torch.cat((x, randomNumber), dim=1)
        
        x1 = F.relu(self.fc1(x1))
        x1 = F.relu(self.fc2(x1))    
        x1 = self.fc3(x1)  

        # Inside the network we usually use relu() as our non-linear activation function, but for the output layer, we use Softmax function for predicting category with maximum probability. 
        #The softmax function returns a positive probability for each of the prediction classes, and the values sum to 1.
        return F.log_softmax(x), F.log_softmax(x1)

In [8]:
!pip install torchsummary
from torchsummary import summary

#This command will return a boolean (True/False) based on the GPU availability.
use_cuda = torch.cuda.is_available()                   #Checking if GPU is available

#use the use_cuda flag to specify which device you want to use
# This will set the device to the GPU if one is available and to the CPU if there isn’t a GPU available. This means that you don’t need to hard code changes into your code to use one or the other.
device = torch.device("cuda" if use_cuda else "cpu") 

#move the model to the choosen device
model = Net().to(device)      

# prints the model summary
#summary(model, [(1, 28, 28),(1,1)])                #Gray scale images hence its only 1 channel

print(model)

Net(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv5): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1))
  (conv6): Conv2d(512, 1024, kernel_size=(3, 3), stride=(1, 1))
  (conv7): Conv2d(1024, 10, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=20, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=30, bias=True)
  (fc3): Linear(in_features=30, out_features=19, bias=True)
)


In [9]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

#load train data and normalize the pixel values with mean and std computed on the MNIST training set
from torch.utils.data import DataLoader
train_loader = DataLoader(train_dataset,batch_size=128,shuffle=True)
test_loader = DataLoader(test_dataset,batch_size=128)

In [10]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target, randomNumber, sum) in enumerate(pbar):
        data, target, randomNumber, sum = data.to(device), target.to(device), randomNumber.to(device), sum.to(device)
         # fetch the data and the target values
        optimizer.zero_grad()
        output, sum_output = model(data, randomNumber)
        MNIST_loss = F.nll_loss(output, target)
        sum_loss = F.nll_loss(sum_output, sum)
        loss = (MNIST_loss + sum_loss)/2
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')

In [11]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    sum_correct = 0
    with torch.no_grad():
        for data, target, randomNumber, sum in test_loader:
            
            data, target, randomNumber, sum = data.to(device), target.to(device), randomNumber.to(device), sum.to(device)
            output, sum_output = model(data, randomNumber)

            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            test_loss += F.nll_loss(sum_output, sum, reduction='sum').item()  # sum up batch loss
            
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

            sum_pred = sum_output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            sum_correct += pred.eq(sum.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average MNIST loss: {:.4f}, MNIST_Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
    print('\nTest set: Average SUM loss: {:.4f}, SUM_Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, sum_correct, len(test_loader.dataset),
        100. * sum_correct / len(test_loader.dataset)))

In [ ]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

epochs = 30
for epoch in range(1, epochs):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

loss=1.2915515899658203 batch_id=0:   0%|          | 1/469 [00:00<01:03,  7.35it/s]


Test set: Average MNIST loss: 2.5588, MNIST_Accuracy: 9566/10000 (96%)


Test set: Average SUM loss: 2.5588, SUM_Accuracy: 932/10000 (9%)



loss=1.0540063381195068 batch_id=0:   0%|          | 1/469 [00:00<01:02,  7.50it/s]


Test set: Average MNIST loss: 2.0593, MNIST_Accuracy: 9807/10000 (98%)


Test set: Average SUM loss: 2.0593, SUM_Accuracy: 984/10000 (10%)



loss=0.6336737275123596 batch_id=0:   0%|          | 1/469 [00:00<00:57,  8.12it/s]

...

loss=0.09403431415557861 batch_id=0:   0%|          | 1/469 [00:00<01:02,  7.52it/s]


Test set: Average MNIST loss: 0.1119, MNIST_Accuracy: 9914/10000 (99%)


Test set: Average SUM loss: 0.1119, SUM_Accuracy: 1008/10000 (10%)



loss=0.033981092274188995 batch_id=0:   0%|          | 1/469 [00:00<00:56,  8.33it/s]


Test set: Average MNIST loss: 0.1172, MNIST_Accuracy: 9912/10000 (99%)


Test set: Average SUM loss: 0.1172, SUM_Accuracy: 970/10000 (10%)



loss=0.01375068910419941 batch_id=178:  38%|███▊      | 179/469 [00:21<00:34,  8.35it/s]

### Inference

In [ ]:
#We load the image
image = test_dataset[0][0] #shape (784,1)
randomNo = torch.Tensor([1])


tensored_image = test_dataset[0][0].unsqueeze(dim=1).to(device)
output_1,output_2 = model(tensored_image,randomNo.to(device))

In [ ]:
print(output_1,output_2)

In [ ]:
pred1 = output_1.argmax(dim=1, keepdim=True)
pred2 = output_2.argmax(dim=1, keepdim=True)

In [ ]:
print(pred1,pred2)

In [ ]:
print(test_dataset[0][1])

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(image.numpy(), cmap=plt.get_cmap('gray'))